Utilizad la librería BeautifulSoup. 


Vais a tener que pensar artimañas para poder coger la información de todas las páginas del catálogo, de todos los libros...

https://books.toscrape.com/index.html

Titulo
Categoría (fantasía, poesía... etc)
Precio
Si está o no en stock
Cuantos quedan
Portada del libro
Descripción
Información variada: UPC, tipo de producto, precio con impuestos, precio sin impuestos, impuestos, reviews


In [7]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs #este alias es standard

In [3]:
#url de la pagina a scrapear
url = 'https://books.toscrape.com/index.html'

html = req.get(url).text #obtenemos el contenido de la pagina
html[:1000] #mostramos los primeros 1000 caracteres

'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:29" />\n        <meta name="description" content="" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shortcut icon" href

In [4]:
# cantidad de caracteres que tiene esta parte del codigo html
len(str(html).split('/head>')[1])

49875

In [5]:
soup = bs(html, 'html.parser') #creamos el objeto soup, y le decimos que lo que le pasamos tiene pinta de html
type(soup)

bs4.BeautifulSoup

In [6]:
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:29" name="created"/>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="static/oscar/favicon.ico" rel="shortcut icon"/>
  <link href="static/oscar/css/styles.css" rel="stylesheet" type="tex

**Búsqueda según IP**

https://tools.keycdn.com/geo?host=137.255.90.7

In [90]:
#Mi url ha cambiado
url_ip = 'https://tools.keycdn.com/geo?host=88.30.60.7'

In [9]:
url_ip='https://tools.keycdn.com/geo?host=137.255.90.7' #tengo parámetros en la direccion en funcion de la IP que busque...🤔

html = req.get(url_ip, headers = headers).content

soup=bs(html, 'html.parser')

In [92]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en" prefix="og: https://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <meta content="57ed0a7e3b29ae8f3a50b7d383902359f80ff602" name="version"/>
  <title>
   IP Location Finder - IP Lookup With Detailed Geolocation Data | KeyCDN Tools
  </title>
  <meta content="What is my IP address? Lookup an IP address or hostname to get the geographical location." name="description"/>
  <meta content="summary_large_image" name="twitter:card"/>
  <meta content="@KeyCDN" name="twitter:site"/>
  <meta content="@KeyCDN" name="twitter:creator"/>
  <meta content="https://tools.keycdn.com/geo?host=137.255.90.7" property="og:url"/>
  <meta content="website" property="og:type"/>
  <meta content="IP Location Finder - IP Lookup With Detailed Geolocation Data | KeyCDN Tools" property="og:title"/>
  <meta content="What is my IP address? Lookup an IP address or hostname to get the geographical 

Es igual que antes, pues la estructura es la misma, primero saco todos los `<dd>`luego todos los `<dt>`

In [93]:
tabla = soup.find('div', {'id': 'geoResult'})
tabla.find_all('dd', {'class': 'col-8 text-monospace'})

[<dd class="col-8 text-monospace">Benin (BJ)</dd>,
 <dd class="col-8 text-monospace">Africa (AF)</dd>,
 <dd class="col-8 text-monospace">9.5 (lat) / 2.25 (long)</dd>,
 <dd class="col-8 text-monospace">2024-09-13 20:13:20 (Africa/Porto-Novo)</dd>,
 <dd class="col-8 text-monospace">137.255.90.7</dd>,
 <dd class="col-8 text-monospace">137.255.90.7</dd>,
 <dd class="col-8 text-monospace">SBIN</dd>,
 <dd class="col-8 text-monospace">28683</dd>]

In [94]:
tabla.find_all('dt')

[<dt class="col-4">Country</dt>,
 <dt class="col-4">Continent</dt>,
 <dt class="col-4">Coordinates</dt>,
 <dt class="col-4">Time</dt>,
 <dt class="col-4">IP address</dt>,
 <dt class="col-4">Hostname</dt>,
 <dt class="col-4">Provider</dt>,
 <dt class="col-4">ASN</dt>]

In [ ]:
#'{:2.2f}'.format(12.5436363636363)  # formato en strings de numeros

Puedo hacer una función que haga de golpe todo lo que hemos hecho hasta ahora con una dirección IP

In [10]:
def geo(ip):

    url = f'https://tools.keycdn.com/geo?host={ip}' #Dinamica en fucnion de la IP que quiero buscar

    html = req.get(url, headers = headers).content

    soup = bs(html, 'html.parser')

    tabla = soup.find('div', {'id': 'geoResult'})

    conexion = [e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]

    return conexion

Vimos que una url también puede tener diferentes parámetros. ¿Qué pasa si tengo una lista de IP de la que quiero obtener la información y hacerme un dataframe?

In [11]:
list_ip = ['137.255.90.7', '195.255.90.10', '177.255.21.7']

Sacad un DataFrame que tenga la información de las direcciones IP contenidas en `list_ip`:
1. probad a hacer vuestra propia funcion
2. haced el scrapeo "manual"
3. hacedlo utilizando la funcion

In [20]:
def geo_completa(ip):

    url = f'https://tools.keycdn.com/geo?host={ip}' #Dinamica en fucnion de la IP que quiero buscar

    html = req.get(url, headers = headers).content

    soup = bs(html, 'html.parser')

    tabla = soup.find('div', {'id': 'geoResult'})

    valores = [e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]
    columnas = [e.text for e in tabla.find_all('dt', {'class': 'col-4'})]

    dicc_respuesta = {k:v for k,v in zip(columnas, valores)}

    return dicc_respuesta

In [22]:
# Inténtalo tú (alumnos)
resultados = []

for ip in list_ip :
    resultados.append(geo_completa(ip))

list

### Resultado con una simple llamada a una función

In [ ]:
# @title
def geolocalizacion_ip(lista):
  #crea una lista vacía en la que va a ir metiendo cada diccionario
  lista_data= []

  #bucle que itera cada elemento de la lista que entra en la función
  for ip in lista:
    url = f'https://tools.keycdn.com/geo?host={ip}' #Dinamica en fucnion de la IP que quiero buscar

    html = req.get(url, headers = user_agent).content

    soup = bs(html, 'html.parser')

    tabla = soup.find('div', {'id': 'geoResult'})

    #hago una lista con la información de cada fila
    informacion = [e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]

    #hago una lista con la información de las columnas (que usaré como keys del diccionario)
    columnas = [e.text for e in tabla.find_all('dt', {'class': 'col-4'})]

    #hago un diccionario que una vez hecho meto en la lista vacía del principio
    data= dict(zip(columnas,informacion))
    lista_data.append(data)

  #fuera del bucle genero el DataFrame con la lista de diccionarios, dando como índices la lista que entra
  #así cada fila tendrá como índice su ip
  df= pd.DataFrame(lista_data, index= lista)


  return df

In [ ]:
geolocalizacion_ip(list_ip).T # con .T pivoto el DataFrame

### RECORDATORIO de como hacer un df desde una lista de diccionarios

In [27]:
import pandas as pd

dicc1 = {'a': 1, 'b': 2, 'c': 3}
dicc2 = {'a': 23, 'b': 52, 'd': 666}
lista_diccionarios = [dicc1, dicc2]

df = pd.DataFrame(lista_diccionarios)
df

,a,b,c,d
0,1,2,3.0,NaN
1,23,52,NaN,666.0


In [29]:
resultados_en_df =  pd.DataFrame(resultados)
resultados_en_df

,Country,Continent,Coordinates,Time,IP address,Hostname,Provider,ASN,City,Region,Postal code
0,Benin (BJ),Africa (AF),9.5 (lat) / 2.25 (long),2024-09-16 19:56:21 (Africa/Porto-Novo),137.255.90.7,137.255.90.7,SBIN,28683,NaN,NaN,NaN
1,Finland (FI),Europe (EU),60.1797 (lat) / 24.9344 (long),2024-09-16 21:56:22 (Europe/Helsinki),195.255.90.10,ws10.lonne.fi,Elisa Oyj,719,Helsinki,Uusimaa (18),00141
2,Colombia (CO),South America (SA),10.4013 (lat) / -75.5267 (long),2024-09-16 13:56:22 (America/Bogota),177.255.21.7,dinamic-tigo-177-255-21-7.tigo.com.co,Colombia Movil,27831,Cartagena,Departamento de Bolívar (BOL),130005


In [30]:
resultados

[{'Country': 'Benin (BJ)',
  'Continent': 'Africa (AF)',
  'Coordinates': '9.5 (lat) / 2.25 (long)',
  'Time': '2024-09-16 19:56:21 (Africa/Porto-Novo)',
  'IP address': '137.255.90.7',
  'Hostname': '137.255.90.7',
  'Provider': 'SBIN',
  'ASN': '28683'},
 {'City': 'Helsinki',
  'Region': 'Uusimaa (18)',
  'Postal code': '00141',
  'Country': 'Finland (FI)',
  'Continent': 'Europe (EU)',
  'Coordinates': '60.1797 (lat) / 24.9344 (long)',
  'Time': '2024-09-16 21:56:22 (Europe/Helsinki)',
  'IP address': '195.255.90.10',
  'Hostname': 'ws10.lonne.fi',
  'Provider': 'Elisa Oyj',
  'ASN': '719'},
 {'City': 'Cartagena',
  'Region': 'Departamento de Bolívar (BOL)',
  'Postal code': '130005',
  'Country': 'Colombia (CO)',
  'Continent': 'South America (SA)',
  'Coordinates': '10.4013 (lat) / -75.5267 (long)',
  'Time': '2024-09-16 13:56:22 (America/Bogota)',
  'IP address': '177.255.21.7',
  'Hostname': 'dinamic-tigo-177-255-21-7.tigo.com.co',
  'Provider': 'Colombia Movil',
  'ASN': '27831

### Ejemplo LinkedIn

In [ ]:
#Vamos intentar scrapear Linkedin
URL='https://www.linkedin.com/jobs/search/' #Esta es la direccion de la pagina de busqueda de empleo de Linkedin general

A esta direccion le podemos poner parametros para afinar nuestra búsqueda.

URL = 'https://www.linkedin.com/jobs/search/?keywords=data&location=Madrid&refresh=true'

`?keywords=data` nos dice que la palabra clave que queremos buscar es 'data'
    
`&location=Madrid` nos dice que queremos buscar en Madrid
    
`&refresh=true` nos dice que queremos que nos devuelva los resultados más recientes

Podemos filtrar más nuestra búsqueda en la web y veremos que van apareciendo más ***parametros*** en la URL.

`&f_TPR=r120960` nos dice que empleos de la ultima semana. El parámetro va en segundos así que 60 * 60 * 24 * número de días atrás que queremos buscar

`&start={i*25}` donde i sería el número de página


In [52]:
#Vamos a intentar scrapear esta URL
import requests as req
from bs4 import BeautifulSoup as bs
url = 'https://www.linkedin.com/jobs/search/?keywords=data&location=Espa%C3%B1a&refresh=true%27&position=1&pageNum=0'
html = req.get(url).content
soup=bs(html, 'html.parser')

In [53]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="d_jobs_guest_search" name="pageKey"/>
  <!-- -->
  <meta content="urlType=jserp_custom;emptyResult=false" name="linkedin:pageTag"/>
  <meta content="en_US" name="locale"/>
  <meta data-app-version="2.0.1974" data-browser-id="0d4d527e-f2e6-4c77-8420-4404d50733b1" data-call-tree-id="AAYiZ70YtKXgy94sKXmLdg==" data-dfp-member-lix-treatment="control" data-disable-jsbeacon-pagekey-suffix="false" data-dna-member-lix-treatment="control" data-enable-page-view-heartbeat-tracking="" data-human-member-lix-treatment="control" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_search;hPBABDyUQIevuSCxVd+QvQ==" data-service-name="jobs-guest-frontend" id="config"/>
  <link href="https://www.linkedin.com/jobs/data-jobs" rel="canonical"/>
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <!-- -->
  <link href="https://static.licdn.com/aero-v1/sc/h/al2o9zrvru7aqj8e1x2rzsrca" rel="icon

In [56]:
# Intentad sacar los ingredientes de la recete de esta sopa y obtener para cada oferta de trabajo:

lista_ofertas = []
#Listado de ofertas
ofertas = soup.find_all('div', {'class':'base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card'})

for oferta in ofertas[:3]:
    dicc_oferta = {} #Diccionario donde almacenamos datos de 1 oferta
    #Titulo oferta
    titulo_oferta = oferta.find('span', {'class':'sr-only'}).text.strip()
    dicc_oferta['titulo_oferta'] = titulo_oferta

    # - Empresa
    empresa = oferta.find('a', {'class':'hidden-nested-link'}).text.strip()
    dicc_oferta['empresa'] = empresa
    
    # - Ubicacion
    ubicacion = oferta.find('span', {'class':'job-search-card__location'}).text.strip()
    dicc_oferta['ubicacion'] = ubicacion
    
    # - Link de la empresa
    link_empresa = oferta.find('a', {'class':'hidden-nested-link'}).attrs['href']
    dicc_oferta['link_empresa'] = link_empresa

    # - Link de la oferta
    link_oferta = oferta.find('a', {'class': 'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]'}).attrs['href']
    dicc_oferta['link_oferta'] = link_oferta
    
    # - Fecha de publicacion
    #Tengo ofertas que no tienen la clase que estamos buscando por lo que cambiamos nuestro código
    #fecha_publicacion = oferta.find('time', {'class':'job-search-card__listdate--new'}).attrs['datetime']
    fecha_publicacion = oferta.find('time').attrs['datetime']
    dicc_oferta['fecha_publicacion'] = fecha_publicacion
    
    
    #Appendeamos a nuestra lista de diccionarios de respuesta
    lista_ofertas.append(dicc_oferta)    

In [57]:
lista_ofertas[0]

{'titulo_oferta': 'Executive Assistant',
 'empresa': 'LinkedIn',
 'ubicacion': 'Mountain View, CA',
 'link_empresa': 'https://www.linkedin.com/company/linkedin?trk=public_jobs_jserp-result_job-search-card-subtitle',
 'link_oferta': 'https://www.linkedin.com/jobs/view/executive-assistant-at-linkedin-4029190592?position=1&pageNum=0&refId=tngTsFdRrVHIjzNfqBNUrg%3D%3D&trackingId=bUDlsPB5WL%2FRiWKoA1qJhA%3D%3D&trk=public_jobs_jserp-result_search-card',
 'fecha_publicacion': '2024-09-18'}

In [45]:
ofertas[1].find('a', {'class': 'base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]'}).attrs['href']

'https://www.linkedin.com/jobs/view/data-analyst-at-opinion-focus-panel-4026276503?position=2&pageNum=0&refId=kfZCYO2147wP1%2FsyH0guAA%3D%3D&trackingId=BB87MzPY5IWzw6%2BlGee0%2Fw%3D%3D&trk=public_jobs_jserp-result_search-card'

In [ ]:
#Os recomiendo ir echando una ojeada al html y la sopa poco a poco para ver como esta estructurado
#Lo primero que tendremos que buscar será el elemento que contiene todas las ofertas....🤔

#PISTA -- base-search-card__info
#De aqui en adelante vosotros solos, tened presente que querremos guardar toda esa info en un dataframe

### REDDIT - https://www.reddit.com/r/Python/

In [58]:
url = 'https://www.reddit.com/r/Python/'
html = req.get(url).content
soup = bs(html, 'html.parser')

In [59]:
print(soup.prettify())

<!DOCTYPE html>
<html class="theme-beta" lang="en-US">
 <head prefix="og: https://ogp.me/ns#">
  <title>
   Reddit - Dive into anything
  </title>
  <meta content="width=device-width, initial-scale=1, viewport-fit=cover" name="viewport"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <meta content="black" name="apple-mobile-web-app-status-bar-style"/>
  <meta content="#000000" name="msapplication-navbutton-color"/>
  <meta content="#000000" name="theme-color"/>
  <meta content="noarchive" name="bingbot">
   <link crossorigin="anonymous" href="https://www.redditstatic.com/" rel="preconnect"/>
   <link href="https://www.redditstatic.com/" rel="dns-prefetch"/>
   <script nonce="yXpmhekKMLL23Jkh7LZAUw==" type="module">
    !function(e,n,o,t,i,r,d){var a=!1,c=!1,f=[],s=function(e){("e"in e||"p"in e||e.f&&e.f.indexOf("capture")>-1||e.f&&e.f.indexOf("withScope")>-1||e.f&&e.f.indexOf("showReportDialog")>-1)&&u(f),s.data.push(e)};function u(n){function o(){import("https://www.re

In [60]:
#Vamos a sacar info de los artículos de la página de Reddit
articulos = soup.find_all('article', {'class':'w-full m-0'})
articulos[0]

<article aria-label="Introducing FastAgency: the fastest way to deploy a console or web app from your AutoGen workflow" class="w-full m-0">
<!-- "data-ks-item" is used by the modqueue-keyboard-shortcuts manager to fetch the item on which to dispatch events to -->
<shreddit-post app="" author="davorrunje" author-id="t2_bt7vm" class="block relative cursor-pointer group bg-neutral-background focus-within:bg-neutral-background-hover hover:bg-neutral-background-hover xs:rounded-[16px] px-md py-2xs my-2xs nd:visible" comment-count="0" content-href="https://www.reddit.com/r/Python/comments/1fjrqui/introducing_fastagency_the_fastest_way_to_deploy/" created-timestamp="2024-09-18T12:37:33.082000+0000" data-ks-item="" domain="self.Python" feedindex="0" icon="https://styles.redditmedia.com/t5_1uq6xv/styles/profileIcon_snooa14098e1-befb-456e-9b83-03c1cfcd321a-headshot.png?width=64&amp;height=64&amp;frame=1&amp;auto=webp&amp;crop=64:64,smart&amp;s=e21a31fa552d404d131009debb69c5ff8eafc63b" id="t3_1fj

In [68]:
#De cada artículo vamos a sacar su autor
autores = [ articulo.find('span', {'class':'whitespace-nowrap'}).text.replace('u/','') for articulo in articulos]
autores
#articulos[0].find('span', {'class':'whitespace-nowrap'}).text.replace('u/','')

['davorrunje', 'jamesdbrock', 'ritchie46']

In [70]:
#Tipo articulo
tipos = [tipo.text.replace('\n','').strip() for tipo in soup.find_all('div',{'class': 'flair-content [&_.flair-image]:align-bottom max-w-full overflow-hidden whitespace-nowrap text-ellipsis'})]
tipos

['Resource', 'Showcase', 'News']

In [76]:
#Vamos a coger la url de cada uno de los artículos
dominio = 'https://www.reddit.com'
urls = [dominio + link.attrs['href'] for link in soup.find_all('a',{'class': 'absolute inset-0'})]
urls


['https://www.reddit.com/r/Python/comments/1fjrqui/introducing_fastagency_the_fastest_way_to_deploy/',
 'https://www.reddit.com/r/Python/comments/1fjjjy0/edifice_is_like_react_but_with_python_instead_of/',
 'https://www.reddit.com/r/Python/comments/1fj0kfi/gpu_acceleration_released_in_polars/']

Cuando tengo una etiqueta sea la que sea dentro tengo su clase, id, y más atributos. href, title, target...

El contenido de los atributos también es accesible.

In [78]:
len(urls)


3